# 필수 모듈 설치

1. 판다스 !pip install pandas
2. 넘파이 !pip install numpy
3. konlpy !pip install konlpy

# 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma
from datetime import datetime

# 데이터 불러오기

In [2]:
df = pd.read_csv(r'C:\Users\ehdgo\OneDrive - 계명대학교\대학\동아리\2023 1,2학기 학술\클롤링\아프니까 사장이다\아프니까 사장이다_하나은행 부터.csv')

In [3]:
df

,Unnamed: 0,keyword,tit,body,comment,date
0,0,하나은행,[[ 소자본 창업으로 효율적인 고마진 메뉴 판매하기~! 간편조리 / 60만원 투자 ...,대.한.민.국 ❤ 국민의 D.N.A\n- 국밥을 '꼭' 판매해야 하는 이유 -\n▼...,wltmdgus123@naver.com 자료 부탁드립니다~!. 찬밧드 냉장고가 없는...,2023.09.15.
1,1,하나은행,어느은행 주로 거래 하시나요?,어느 은행 주로 이용하시나요?\n그래도 좀 더 믿을만 하고 고객입장 배려해주는?\n...,"주거래는 농협, 뱅킹이나 적금 이런건 카카오랑 토스 사용해요~. 하나은행으로 갈아 ...",2023.09.15.
2,2,하나은행,대출 받을 때 은행,이번에 신용보증재단에서\n특례 대출 신청했는데요.\n하나은행이랑 우리은행 두개 중에...,상관없어유..... 기대출은 어느 은행에서나 조회되서 상관 없어요. 요즘 은행 이자...,2023.09.15.
3,3,하나은행,쿠팡도 네이버처럼 판매자에게 빠른 판매대금정산 도입한다,쿠팡이 판매자들한테\n던지는 불리한 카드 중에\n느린 정산은 늘 손꼽혀왔는데\n대금...,쿠팡은 진짜 정산시스템 좀 바꿔줬으면 했는데 희소식이네요!! 빨리 진행되길!!. 카...,2023.09.15.
4,4,하나은행,수탁보증 거치기간연장 해보신분 계실까요?,하나은행에서 수탁보증 2000받았었는데\n올해 8월부터 원금상환이 시작되었고\n원금...,신용보증기금 수탁보증 대출이시면 2년거치 3년상환이 3년거치 2년상환으로 바뀝니다....,2023.09.15.
...,...,...,...,...,...,...
3880,3880,신한카드,자영업자 님들 식자재 살때 쓰는 카드 사용 머하세요 ㅎ,카드값이 평균 600만원 쓰는데 무슨 카드가 좋을까요 \n 지금은 현대카드m3 사용...,저는 마일리지 많이 쌓이는 카드로 열심히 긁고있습니다~ 목표는 기족과 유럽여행갈때 ...,2019.03.26.
3881,3881,신한카드,카드론에서 2천 대출 받았는데,신한카드론에서 2천만원 대출 받았어요 \n\n처음엔 13.7프로 라고 이야기 하...,카드론있음힘든걸로알아요. 카드론 하기전에 하셨어야 되요. 카드론 쓴 순간부터 왠만한...,2018.11.20.
3882,3882,신한카드,손님이 카드 교환해달라고 할때요,몇일 전에 손님이 신한 카드로 결제 했는데요\n오늘 와서니 국민카드로 교환해달라고 ...,오늘매출입니다. 재결제 햇는데 왜 매출이 안뜨죠?.,2018.10.19.
3883,3883,신한카드,경북울진 연매출 3억5천~4억내외 레스토랑하실분 찾아요!,"\n <header class=""header fix"" id=""hd"" style...",댓글 없음,2018.08.08.


# 전처리

In [4]:
# 필요없는 열 제거
df = df[['keyword', 'tit', 'body', 'comment', 'date']]

# 중복값 지우기
df = df.drop_duplicates(subset=['tit', 'body'])

# 결측치 채우기
df['body'] = df['body'].fillna('')

# tit, body, comment 합치기, 컬럼 삭제
df.loc[:,'text'] = df['tit'] + ' ' + df['body'] + ' ' + df['comment']
df=df.drop(['tit','body','comment'],axis=1)

# 불용어 처리 함수
def preprocess_text(text):
    if isinstance(text, str):
        # HTML/XML 태그 제거
        text = re.sub(r'<[^>]+>', '', text)
        # 개행 문자를 공백으로 대체
        text = re.sub(r'\n', ' ', text)
        # 연속된 공백 문자를 하나의 공백으로 대체
        text = re.sub(r'\s+', ' ', text)
        # '갤'을 '개월'로 대체
        text = re.sub(r'[0-9]+갤', '개월', text)
        # 한글과 온점을 제외한 모든 문자 제거
        text = re.sub(r'[^가-힣.]', ' ', text)
        # '.' 다음에 나오는 공백을 없애기
        text = re.sub(r'\.\s+', '.', text)
        # 연속된 '.'을 하나의 '.'으로 대체
        text = re.sub(r'\.{2,}', '.', text)
        # '.' 다음에 나오는 공백을 없애기
        text = re.sub(r'\.\s+', '.', text)
        # 연속된 공백 문자를 하나의 공백으로 대체(한번 더)
        text = re.sub(r'\s+', ' ', text)   
    return text
# 불용어 처리
df.loc[:, 'text'] = df['text'].apply(preprocess_text)

# 특정 단어가 들어간 행 삭제(ex. 광고, 홍보글 등등)
words_to_delete = ['(협찬)', '체험단', '무료체험', '체험후기', '예약링크']
pattern = '|'.join(words_to_delete)
delete = df['text'].str.contains(pattern)
df.drop(df.index[delete], inplace = True)
df = df.reset_index(drop=True)

# 반복 문장 삭제
df.text = df.text.str.replace("긴글+ 사진폭탄 주의","")
df.text = df.text.str.replace("질문 있습니당.","")
df.text = df.text.str.replace("★","")
df.text = df.text.str.replace("댓글 없음","")
df.text = df.text.str.replace("\*주사기, 수액, 처방약 등 의약품 등록 불가* '채팅 거래 불가","")
df.text = df.text.str.replace("채팅 거래 불가 / 글 등록 후 채팅으로 연락오는 대부분이 사기며 모든 거래는 메모인증 등 실제 제품 확인해야 합니다. 채팅거래는 '사기치세요'와 같은 뜻입니다.","")
df.text = df.text.str.replace("\* 미준수 글은 활동정지/게시글 삭제","")
df.text = df.text.str.replace("안녕하세요","")
df.text = df.text.str.replace("안녕하세요!","")
df.text = df.text.str.replace("안녕하세요.","")
df.text = df.text.str.replace("감사합니다.","")
df.text = df.text.str.replace("챗드릴게요","")
df.text = df.text.str.replace("쪽지주세요","")
df.text = df.text.str.replace("쪽지드릴게요","")
df.text = df.text.str.replace("쪽지 드렸어요","")
df.text = df.text.str.replace("\* 규정 지킬 생각이 없으면 다른 카페에서 거래해 주세요.","")
df.text = df.text.str.replace("1\)메모인증은 수기작성(종이에 직접 아이디/닉네임/판매날짜/제품명 제품수량 제품용량)기입","")
df.text = df.text.str.replace("2\)작성 후 직접 촬영한 제품 사진과 함께 보이도록 사진 두 장 필수(제품 전체사진+메모인증 보이게 1장, 메모 잘 보이는 사진 1장) (상세내용은 배너클릭) ","")
df.text = df.text.str.replace("3\)판매할 제품은 모두 사진찍어 수량/용량 정확히 기입. ","")
df.text = df.text.str.replace("4\)벼룩 판매 가격규정 미준수 거래완료 판매자 아이디는 영구활동정지되니 반드시 공지 확인후 글등록","")
df.text = df.text.str.replace("완료되었습니다~.","")
df.text = df.text.str.replace("나눔후기\)","")
df.text = df.text.str.replace("\[체험후기\]","")
# 띄어쓰기 한번 더 처리
def re_spacing(text):
    result = text.strip()
    while "  " in result:
        result = result.replace("  ", " ")
    return result
df['text'] = df['text'].apply(lambda x : re_spacing(x))

# 형태소 분석 함수 지정
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Komoran()
    elif tokenizer_name == "Mecab":
        tokenizer = Komoran()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    else:
        tokenizer = Okt()
    return tokenizer
# 형태소 분석 진행
tokenizer = get_tokenizer(Mecab)
def pos_tagging_and_filter(text):
    # Mecab을 사용하여 텍스트의 품사 태깅을 수행합니다.
    pos_tokens = tokenizer.pos(text)
    # 'Josa' 품사를 가진 토큰(조사)을 필터링합니다.
    filtered_tokens = [token for token, pos in pos_tokens if pos != 'Josa']
    # 추가적인 단어들을 필터링합니다.
    filtered_tokens = [token for token in filtered_tokens if token not in ['그리고', '그러나', '그런데', '그래서', '또는', '혹은']]
    return filtered_tokens
df['tok_text'] = df['text'].apply(pos_tagging_and_filter)
#형태소 분석파일 저장 - 형태소 분석 완료된 시간으로 파일명
now = datetime.now()  
formatted_time = now.strftime("%Y%m%d_%H%M%S") 
filename = f"형태소분석_{formatted_time}.csv" 
df.to_csv(filename, index=False)

C:\Users\ehdgo\AppData\Local\Temp\ipykernel_37416\4116603124.py:42: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  delete = df['text'].str.contains(pattern)


In [5]:
df

,keyword,date,text,tok_text
0,하나은행,2023.09.15.,소자본 창업으로 효율적인 고마진 메뉴 판매하기 간편조리 만원 투자 낮은 원가 고객 ...,"[소자본, 창업, 효율, 적, 고마진, 메뉴, 판매, 하기, 간편, 조리, 만원, ..."
1,하나은행,2023.09.15.,어느은행 주로 거래 하시나요 어느 은행 주로 이용하시나요 그래도 좀 더 믿을만 하고...,"[어느, 은행, 주로, 거래, 하시나요, 어느, 은행, 주로, 이용, 하시나요, 그..."
2,하나은행,2023.09.15.,대출 받을 때 은행 이번에 신용보증재단에서 특례 대출 신청했는데요.하나은행이랑 우리...,"[대출, 받을, 때, 은행, 이번, 신용, 보증, 재단, 특례, 대출, 신청, 했는..."
3,하나은행,2023.09.15.,쿠팡도 네이버처럼 판매자에게 빠른 판매대금정산 도입한다 쿠팡이 판매자들한테 던지는 ...,"[쿠팡, 네이버, 판매, 자, 빠른, 판매, 대, 금정산, 도입, 한다, 쿠팡, 판..."
4,하나은행,2023.09.15.,수탁보증 거치기간연장 해보신분 계실까요 하나은행에서 수탁보증 받았었는데 올해 월부터...,"[수탁, 보증, 거치, 기간, 연장, 해보, 신분, 계실까, 요, 하나은행, 수탁,..."
...,...,...,...,...
2557,신한카드,2019.03.26.,자영업자 님들 식자재 살때 쓰는 카드 사용 머하세요 카드값이 평균 만원 쓰는데 무슨...,"[자영, 업자, 님들, 식, 자재, 살때, 쓰는, 카드, 사용, 머, 하세요, 카드..."
2558,신한카드,2018.11.20.,카드론에서 천 대출 받았는데 신한카드론에서 천만원 대출 받았어요 처음엔 .프로 라고...,"[카드, 론, 천, 대출, 받았는데, 신한카드, 론, 천만원, 대출, 받았어요, 처..."
2559,신한카드,2018.10.19.,손님이 카드 교환해달라고 할때요 몇일 전에 손님이 신한 카드로 결제 했는데요 오늘 ...,"[손님, 카드, 교환, 해달라고, 할, 때, 몇, 일, 전, 손님, 신한, 카드, ..."
2560,신한카드,2018.08.08.,경북울진 연매출 억 천 억내외 레스토랑하실분 찾아요 카페홈 아프니까 사장이다 검색 ...,"[경북, 울진, 연매출, 억, 천, 억, 내외, 레스토랑, 하실, 분, 찾아요, 카..."
